First, load the dataset from the weatherinszeged table from Thinkful's database.

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

In [26]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
dff = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [27]:
dff.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


Build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed.

In [28]:
#Create X and Y variables
Y = dff['temperature']
X = dff[['humidity', 'windspeed']]
#add constant to X with statistics model
X = sm.add_constant(X)
#we apply ordinary least square to our variables
results = sm.OLS(Y, X).fit()
#print results
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            temperature   R-squared:                       0.418
Model:                            OLS   Adj. R-squared:                  0.418
Method:                 Least Squares   F-statistic:                 3.470e+04
Date:                Tue, 20 Oct 2020   Prob (F-statistic):               0.00
Time:                        17:25:31   Log-Likelihood:            -3.2838e+05
No. Observations:               96453   AIC:                         6.568e+05
Df Residuals:                   96450   BIC:                         6.568e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         37.8674      0.108    350.251      0.000      37.656      38.079
humidity     -32.4374      0.123   -263.415      0.000     -32.679     -32.196
windspeed     -0.1939      0.003    -55.701      0.000      -0.201      -0.187
==============================================================================
Omnibus:                     3415.546   Durbin-Watson:                   0.056
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3844.707
Skew:                          -0.458   Prob(JB):                         0.00
Kurtosis:                       3.344   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Are the estimated coefficients statistically significant? Yes, both of them
Are the signs of the estimated coefficients in line with your previous expectations? I had no expectations, but they look reasonable.
Interpret the estimated coefficients. As humididty or windspeed increase, the differnce between appearent temp and temp decrease.
What are the relations between the target and the explanatory variables?

Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. 

In [30]:
#create interaction variable 
dff['interaction_humidity_windspeed'] = dff['humidity'] * dff['windspeed']
#Create X and Y variables
Y = dff['temperature']
X = dff[['humidity', 'windspeed', 'interaction_humidity_windspeed']]
#add constant to X with statistics model
X = sm.add_constant(X)
#we apply ordinary least square to our variables
results = sm.OLS(Y, X).fit()
#print results
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            temperature   R-squared:                       0.419
Model:                            OLS   Adj. R-squared:                  0.419
Method:                 Least Squares   F-statistic:                 2.317e+04
Date:                Tue, 20 Oct 2020   Prob (F-statistic):               0.00
Time:                        17:26:34   Log-Likelihood:            -3.2836e+05
No. Observations:               96453   AIC:                         6.567e+05
Df Residuals:                   96449   BIC:                         6.568e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             38.9435      0.179    218.014      0.000      38.593      39.294
humidity                         -33.9031      0.230   -147.708      0.000     -34.353     -33.453
windspeed                         -0.2898      0.013    -22.053      0.000      -0.316      -0.264
interaction_humidity_windspeed     0.1358      0.018      7.566      0.000       0.101       0.171
==============================================================================
Omnibus:                     3396.212   Durbin-Watson:                   0.056
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3831.009
Skew:                          -0.454   Prob(JB):                         0.00
Kurtosis:                       3.357   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Are the coefficients statistically significant? Yes, all three of them.
Did the signs of the estimated coefficients for humidity and windspeed change? Yes, they changed to positve. Also they have signifficantly lower values.
Interpret the estimated coefficients. ??

Run your house prices model again 

In [17]:
#load data
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [21]:
#get dummies
df = pd.concat([df,pd.get_dummies(df.mszoning, prefix="mszoning", drop_first=True)], axis=1)
df = pd.concat([df,pd.get_dummies(df.street, prefix="street", drop_first=True)], axis=1)

In [22]:
# create variable X, Y
Y = df['saleprice']
X = df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf']]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.761
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     926.5
Date:                Tue, 20 Oct 2020   Prob (F-statistic):               0.00
Time:                        16:31:00   Log-Likelihood:                -17499.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1454   BIC:                         3.504e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -9.907e+04   4638.450    -21.359      0.000   -1.08e+05      -9e+04
overallqual  2.364e+04   1072.532     22.037      0.000    2.15e+04    2.57e+04
grlivarea      45.3458      2.489     18.218      0.000      40.463      50.228
garagecars   1.454e+04   3022.681      4.812      0.000    8615.034    2.05e+04
garagearea     17.1334     10.468      1.637      0.102      -3.401      37.668
totalbsmtsf    31.5015      2.904     10.848      0.000      25.805      37.198
==============================================================================
Omnibus:                      438.146   Durbin-Watson:                   1.971
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            40648.199
Skew:                          -0.338   Prob(JB):                         0.00
Kurtosis:                      28.841   Cond. No.                     9.23e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.23e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Which features are statistically significant, and which are not? Everything is stat signifficant, except 